## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Vaini,TO,-21.2000,-175.2000,77.00,73,20,17.27,['few clouds']
1,Albany,US,42.6001,-73.9662,63.00,40,99,3.00,['overcast clouds']
2,Cherskiy,RU,68.7500,161.3000,24.66,97,92,3.33,['overcast clouds']
3,Dikson,RU,73.5069,80.5464,15.84,97,61,11.32,['broken clouds']
4,Marang,MY,5.2056,103.2059,77.00,100,20,2.30,['few clouds']


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 100


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Vaini,TO,-21.2000,-175.2000,77.00,73,20,17.27,['few clouds']
4,Marang,MY,5.2056,103.2059,77.00,100,20,2.30,['few clouds']
5,Mareeba,AU,-17.0000,145.4333,71.60,78,47,10.36,['scattered clouds']
7,Rikitea,PF,-23.1203,-134.9692,78.44,79,69,13.22,['broken clouds']
19,Mahebourg,MU,-20.4081,57.7000,73.40,83,40,5.75,['light rain']
21,Avarua,CK,-21.2078,-159.7750,75.20,69,90,9.22,['overcast clouds']
25,Cotonou,BJ,6.3654,2.4183,84.20,79,40,13.80,['scattered clouds']
27,Chodavaram,IN,17.8333,82.9500,84.20,84,40,2.30,"['moderate rain', 'mist']"
31,San Cristobal,VE,7.7669,-72.2250,80.60,69,40,8.05,['scattered clouds']
32,Tessalit,ML,20.1986,1.0114,92.89,15,23,12.28,['few clouds']


In [10]:
# 4a. Determine if there are any empty rows.
for column in preferred_cities_df.columns:
    print(f"Column {column} has {preferred_cities_df[column].isnull().sum()} null values")

Column City has 0 null values
Column Country has 1 null values
Column Lat has 0 null values
Column Lng has 0 null values
Column Max Temp has 0 null values
Column Humidity has 0 null values
Column Cloudiness has 0 null values
Column Wind Speed has 0 null values
Column Current Description has 0 null values


In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vacation_cities = preferred_cities_df.dropna()

In [12]:
for column in vacation_cities.columns:
    print(f"Column {column} has {vacation_cities[column].isnull().sum()} null values")

Column City has 0 null values
Column Country has 0 null values
Column Lat has 0 null values
Column Lng has 0 null values
Column Max Temp has 0 null values
Column Humidity has 0 null values
Column Cloudiness has 0 null values
Column Wind Speed has 0 null values
Column Current Description has 0 null values


In [13]:
vacation_cities.count()

City                   285
Country                285
Lat                    285
Lng                    285
Max Temp               285
Humidity               285
Cloudiness             285
Wind Speed             285
Current Description    285
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,77.00,['few clouds'],-21.2000,-175.2000,
4,Marang,MY,77.00,['few clouds'],5.2056,103.2059,
5,Mareeba,AU,71.60,['scattered clouds'],-17.0000,145.4333,
7,Rikitea,PF,78.44,['broken clouds'],-23.1203,-134.9692,
19,Mahebourg,MU,73.40,['light rain'],-20.4081,57.7000,
21,Avarua,CK,75.20,['overcast clouds'],-21.2078,-159.7750,
25,Cotonou,BJ,84.20,['scattered clouds'],6.3654,2.4183,
27,Chodavaram,IN,84.20,"['moderate rain', 'mist']",17.8333,82.9500,
31,San Cristobal,VE,80.60,['scattered clouds'],7.7669,-72.2250,
32,Tessalit,ML,92.89,['few clouds'],20.1986,1.0114,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,77.00,['few clouds'],-21.2000,-175.2000,Keleti Beach Resort
4,Marang,MY,77.00,['few clouds'],5.2056,103.2059,DBarakah Homestay
5,Mareeba,AU,71.60,['scattered clouds'],-17.0000,145.4333,Trinity Plains Tourist Park
7,Rikitea,PF,78.44,['broken clouds'],-23.1203,-134.9692,Pension Maro'i
19,Mahebourg,MU,73.40,['light rain'],-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
...,...,...,...,...,...,...,...
710,Carlyle,US,71.01,['scattered clouds'],38.6103,-89.3726,Mariner's Village at Carlyle Lake
711,Pangai,TO,78.80,['scattered clouds'],-19.8000,-174.3500,Ha'apai Beach Resort
715,Manokwari,ID,78.85,['overcast clouds'],-0.8667,134.0833,Swiss Belhotel Manokwari
716,Madaoua,NE,90.28,['few clouds'],14.0730,5.9600,


In [17]:
# 7. Drop the rows where there is no Hotel Name.
for column in hotel_df.columns:
    print(f"Column {column} has {hotel_df[column].isnull().sum()} null values")

Column City has 0 null values
Column Country has 0 null values
Column Max Temp has 0 null values
Column Current Description has 0 null values
Column Lat has 0 null values
Column Lng has 0 null values
Column Hotel Name has 0 null values


In [ ]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
